<a href="https://colab.research.google.com/github/AmanBRajani/Episcan-AI-Models/blob/main/Dark_Circle_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ STEP 0: Fix Colab Locale Bug
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

# ✅ STEP 1: Install Required Libraries
!pip install --upgrade ultralytics opencv-python-headless pycocotools kaggle

# ✅ STEP 2: Import Required Libraries
import shutil
import glob
import yaml
import json
import torch
import numpy as np
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

# ✅ STEP 3: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ STEP 4: Kaggle API Setup
if not os.path.exists("/root/.kaggle/kaggle.json"):
    from google.colab import files
    print("⚠️ Upload kaggle.json to proceed.")
    files.upload()
    os.makedirs("/root/.kaggle", exist_ok=True)
    shutil.move("kaggle.json", "/root/.kaggle/")
    os.chmod("/root/.kaggle/kaggle.json", 600)

# ✅ STEP 5: Download & Extract Acne Dataset Only (Remove Open Pores Datasets)
# Acne Datasets
!kaggle datasets download -d osmankagankurnaz/acne-dataset-in-yolov8-format --unzip -p /content/acne_dataset
# Second Acne Dataset
!kaggle datasets download -d anand-itt/acne-detection --unzip -p /content/acne_dataset2

# ✅ STEP 6: Verify Downloads for Acne Datasets
# Check if datasets exist and print contents
def check_dataset_path(path, dataset_name):
    if os.path.exists(path):
        print(f"📁 {dataset_name} dataset:", os.listdir(path))
    else:
        print(f"❗ {dataset_name} dataset not found! Please check the path and ensure the dataset is downloaded.")

# Define dataset paths for acne datasets
acne_dataset1_path = '/content/acne_dataset/data-2'
acne_dataset2_path = '/content/acne_dataset2'

check_dataset_path(acne_dataset1_path, 'Acne Dataset 1')
check_dataset_path(acne_dataset2_path, 'Acne Dataset 2')

# ✅ STEP 7: Prepare Folder Structure for Acne Datasets
splits = ['train', 'valid', 'test']

# Create necessary folders for acne datasets only
for dataset in [acne_dataset1_path, acne_dataset2_path]:
    for split in splits:
        os.makedirs(f'{dataset}/{split}/images', exist_ok=True)
        os.makedirs(f'{dataset}/{split}/labels', exist_ok=True)

# ✅ STEP 8: Move Images to YOLO Folder Structure
# Move images into the correct structure for YOLOv8 training (for acne datasets only)
for dataset in [acne_dataset1_path, acne_dataset2_path]:
    for split in splits:
        img_files = glob.glob(f'{dataset}/{split}/*.jpg')
        for img in img_files:
            shutil.move(img, f'{dataset}/{split}/images/{os.path.basename(img)}')

# ✅ STEP 9: Convert COCO Annotations to YOLO Labels for Acne Datasets
# Convert COCO annotations to YOLO format for each acne dataset
def convert_coco_to_yolo(coco_json_path, labels_dir):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    category_id_to_label = {cat['id']: 1 for cat in coco_data['categories']}  # acne = class 1
    images_info = {img['id']: img for img in coco_data['images']}
    annotations_by_image = {}

    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        bbox = ann['bbox']
        category_id = ann['category_id']
        annotations_by_image.setdefault(image_id, []).append((category_id, bbox))

    for image_id, anns in tqdm(annotations_by_image.items(), desc=f"🔄 Converting {os.path.basename(coco_json_path)}"):
        img_info = images_info[image_id]
        img_width = img_info['width']
        img_height = img_info['height']
        label_lines = []

        for category_id, bbox in anns:
            label = category_id_to_label[category_id]
            x, y, w, h = bbox
            x_center = (x + w / 2) / img_width
            y_center = (y + h / 2) / img_height
            w /= img_width
            h /= img_height
            label_lines.append(f"{label} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

        label_filename = os.path.splitext(img_info['file_name'])[0] + ".txt"
        with open(os.path.join(labels_dir, label_filename), 'w') as f:
            f.write("\n".join(label_lines))

# Only process acne datasets
for dataset in [acne_dataset1_path, acne_dataset2_path]:
    for split in splits:
        coco_json = f'{dataset}/{split}/_annotations.coco.json'
        labels_dir = f'{dataset}/{split}/labels'
        if os.path.exists(coco_json):
            convert_coco_to_yolo(coco_json, labels_dir)
        else:
            print(f"❗ COCO JSON not found for {split} in {dataset}")

# ✅ STEP 10: Train Each Acne Dataset Individually
def train_dataset(dataset_path, num_classes, class_names):
    data_yaml = {
        'path': dataset_path,
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'nc': num_classes,  # Number of classes
        'names': class_names  # Class names list
    }

    yaml_path = f'{dataset_path}/data.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    print(f"✅ {dataset_path} data.yaml ready!")

    # Check GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"✅ Training on: {device}")

    # YOLOv8 Training (High-Res)
    model = YOLO('yolov8n.pt')  # Using YOLOv8 Nano model for testing. Change this to yolov8s.pt if issues persist.

    # Start Training
    results = model.train(
        data=yaml_path,
        epochs=5,  # Reduced epochs for testing
        imgsz=640,  # Reduced image size for testing
        batch=4 if device == 'cuda' else 2,  # Adjust batch size
        optimizer='SGD',
        lr0=0.001,
        patience=30,
        val=True,
        device=device,
        pretrained=True,
        close_mosaic=1,
        resume=False,
        workers=2,
        project='/content/drive/MyDrive/yolov8_project',
        name=f'{dataset_path}_detector'
    )

# ✅ STEP 11: Save Final Model to Drive
# The model weights are saved to the directory after training
trained_weights = f"{results.save_dir}/weights/best.pt"  # Get the best model weights path

# Ensure the target directory in Google Drive exists
save_path = '/content/drive/MyDrive/yolov8_project_final_model.pt'  # Update to your desired final save path

# Copy the best model weights to Google Drive
shutil.copy(trained_weights, save_path)
print(f"✅ Model saved to: {save_path}")

return save_path


# Train each acne dataset individually and save the model paths
acne_dataset_save_path = train_dataset(acne_dataset1_path, 1, ['acne'])
acne_dataset2_save_path = train_dataset(acne_dataset2_path, 1, ['acne'])

# ✅ STEP 12: Test Models on Images (Optional)
def detect_and_display(image_url, model_path):
    import requests
    from PIL import Image
    from io import BytesIO

    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img_np = np.array(img)

    model = YOLO(model_path)
    results = model.predict(img_np, imgsz=640, conf=0.3)  # Reduced image size

    for r in results:
        for box, cls in zip(r.boxes.xyxy, r.boxes.cls):
            x1, y1, x2, y2 = map(int, box)
            label = data_yaml['names'][int(cls)]
            cv2.rectangle(img_np, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img_np, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the result
    plt.imshow(img_np)
    plt.show()

# Example: Detect on an Image from the web
detect_and_display("https://t3.ftcdn.net/jpg/01/98/12/08/360_F_198120802_sgLvDf4N5AGIPQUbBH5lvG8ylYrHMG64.jpg", acne_dataset_save_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 